In [41]:
import warnings
warnings.filterwarnings("ignore")

# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Milvus

In [11]:
from langchain_community.vectorstores import Milvus
# from langchain_milvus import MilvusVectorStore 
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

In [5]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

[Document(metadata={'pk': 460125151305459458, 'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 13}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [6]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', 
                model='qwen3:8B', 
                temperature=0.1)

my_llm.invoke("你好/no_think")

/tmp/ipykernel_935149/307156435.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434',


'<think>\n\n</think>\n\n你好！😊 有什么我可以帮助你的吗？'

## 3. 构建检索问答链

prompts

In [7]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question", "sources"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [8]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [9]:
question_1 = "什么是ZXVMAX-S的5GC业务？no_think"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
# print(result["source_documents"])

/tmp/ipykernel_935149/3076973797.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question_1})


大模型+知识库后回答 question_1 的结果：
<think>

</think>

ZXVMAX-S的5GC业务是指ZXVMAX-S多维价值分析系统针对5G核心网（5GC）场景所提供的网络运维和运营分析服务。该系统支持对5G网络中的各种业务和性能进行深入分析，包括但不限于网络质量分析、用户感知分析、问题根因定位、业务优化等。它能够帮助运营商实时监控和优化5G网络性能，提升用户体验，并为网络建设、运维优化和客户服务提供全面支撑。谢谢你的提问！


In [10]:
question_2 = "严威是谁？no_think"
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
<think>

</think>

我是一个AI助手，无法提供关于“严威”的具体信息。谢谢你的提问！


### 4.2 无知识库大模型自己回答的效果

In [49]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

'<think>\n\n</think>\n\nZXVMAX-S 是华为推出的一款 5G 核心网（5GC）业务平台，主要用于提供 5G 网络中的核心网功能，支持多种 5G 业务场景。ZXVMAX-S 的全称是 **ZXVMAX-S 5G Core Business Platform**，它是一个基于云原生架构的 5G 核心网业务平台，旨在为运营商提供灵活、高效、可扩展的 5G 核心网解决方案。\n\n### ZXVMAX-S 的 5GC 业务主要包括以下内容：\n\n1. **5G 核心网功能支持**：\n   - 支持 5G 网络中的关键功能模块，如 **AMF（接入和移动性管理功能）**、**SMF（会话管理功能）**、**UPF（用户面功能）**、**AUSF（认证服务器功能）**、**UDMF（统一数据管理功能）** 等。\n   - 支持 5G 网络切片、网络能力开放、服务化架构（Service-Based Architecture, SBA）等。\n\n2. **多业务场景支持**：\n   - 支持 eMBB（增强移动宽带）、URLLC（超可靠低时延通信）、mMTC（大规模机器类通信）等 5G 三大典型业务场景。\n   - 支持 5G 与 4G/3G/2G 网络的互操作，实现多网络融合。\n\n3. **云原生架构**：\n   - 基于云原生技术（如容器化、微服务、Kubernetes 等），实现灵活部署、弹性扩展和高可用性。\n   - 支持 NFV（网络功能虚拟化）和 SDN（软件定义网络）技术，提升网络灵活性和智能化水平。\n\n4. **开放能力平台**：\n   - 提供开放 API 接口，支持与第三方应用、行业应用、合作伙伴的对接，推动 5G 网络能力开放和生态建设。\n   - 支持网络切片管理、业务链编排、网络自动化运维等功能。\n\n5. **安全与可靠性**：\n   - 提供全面的安全机制，包括用户身份认证、数据加密、访问控制等，保障 5G 网络的安全性。\n   - 支持高可用性架构，确保业务连续性和网络稳定性。\n\n### 总结：\nZXVMAX-S 是华为面向 5G 核心网业务的综合解决方案，支持 5G 网络中的关键功能模块和多种业务场景，具备云原生架构、开放能力、高安全性和高可靠性，旨在帮助运营商构建灵活、高效、可扩

In [50]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

'<think>\n\n</think>\n\n“严威”是一个常见的中文名字，可能指不同领域的人。以下是几种可能的解释：\n\n1. **历史人物**：  \n   严威是东汉时期的人物，字子高，是严光（即严子陵）的弟弟。严光是东汉开国功臣刘秀的故交，以隐士著称。严威在历史上记载较少，但可能与严光有某种关联。\n\n2. **现代人物**：  \n   - **严威（演员）**：中国内地男演员，参演过《大宅门》《新三国》等电视剧。  \n   - **严威（作家/学者）**：可能指某些学者或作家，但需要更多背景信息确认。  \n   - **其他领域**：可能指企业高管、运动员等，但需具体上下文。\n\n3. **其他可能性**：  \n   - 可能是某个地方的名人、家族成员，或网络上的昵称。\n\n**总结**：  \n若需准确回答“严威是谁”，需要更多上下文（如领域、时代、相关事件等）。若指历史人物，可能与东汉严光有关；若指现代人物，可能是演员或学者。建议补充信息以进一步确认。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [38]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [39]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

In [40]:
questions = [
    "什么是ZXVMAX-S的5GC业务？no_think",
    "包含哪些功能no_think", 
    "整理成表格no_think"  
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

回答：<think>

</think>

ZXVMAX-S的5GC业务是指ZXVMAX-S多维价值分析系统针对5G核心网（5GC）场景所提供的网络运维和运营分析服务。该系统能够支持5G网络中的信令采集、数据存储、分析处理以及多维度的业务洞察，帮助运营商实现对5G网络质量、用户行为、终端性能、业务特征等方面的全面分析和优化。ZXVMAX-S在5GC业务中，能够支持实时分析和事后分析两种模式，为5G网络的运维和运营提供全面支撑。

谢谢你的提问！


回答：<think>

</think>

ZXVMAX-S的5GC业务包含以下主要功能：

1. **一键投诉功能**  
   输入用户号码或IMSI以及投诉的起止时间，系统可以快速定位引起用户投诉的具体问题业务记录（即问题话单），并分析导致问题发生的原因，包括4G、5G无线、核心网、SP以及用户等因素。

2. **区域感知功能**  
   定期（支持天/周/月粒度）对全网的所有小区进行排查，识别感知较差的小区，并分析导致这些小区用户感知差的原因，涵盖4G、5G无线、核心网、SP以及用户等因素。同时支持对事先配置的重点区域进行感知分析，包括感知评估和根因定位。

3. **用户感知功能**  
   定期（支持天/周/月粒度）对全网的所有用户进行排查，识别感知较差的用户，并分析导致这些用户感知差的原因，涵盖4G、5G无线、核心网、SP以及用户等因素。同时支持对事先配置的VIP用户进行感知分析，包括感知评估和根因定位。

4. **实时分析与事后分析**  
   ZXVMAX-S支持两种分析模式：  
   - **实时分析**：用于网络质量、数据业务质量等的实时监控和分析。  
   - **事后分析**：用于用户投诉、问题分析等场景的深度挖掘和根因定位。

5. **多维度分析能力**  
   系统能够从网元、用户、终端、业务等多个维度对网络和业务信息进行全方位分析，支持性能优化、事件处理、用户分析、终端分析、业务优化等业务功能。

6. **灵活的数据查询与分析**  
   支持根据用户需求进行数据自定义查询或分析，满足不同场景下的分析需求。

这些功能共同支撑了5G核心网（5GC）的运维和运营分析，帮助运营商提升网络质量、优化用户体验，并挖掘数据价值。

谢谢你的提问！


回答：<think>

<